In [ ]:
# combine all the multiome samples together

In [ ]:
setwd(file.path("/volume/afib/out/"))

In [ ]:
library(Seurat)
library(dplyr)
library(ggplot2)
library(Signac)
library(GenomeInfoDb)
library(BSgenome.Hsapiens.UCSC.hg38)
library(EnsDb.Hsapiens.v86) # for a set of annotations for hg38

# Load in data

In [ ]:
ctrl1 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_1_LA_postDoublet+RNAcontamFilt.rds')
ctrl2 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_2_LA_postDoublet+RNAcontamFilt.rds')
ctrl3 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_3_LA_postDoublet+RNAcontamFilt.rds')
ctrl4 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_4_LA_postDoublet+RNAcontamFilt.rds')
ctrl5 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_5_LA_postDoublet+RNAcontamFilt.rds')
ctrl6 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_6_LA_postDoublet+RNAcontamFilt.rds')
ctrl7 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_7_LA_postDoublet+RNAcontamFilt.rds')
ctrl8 <- readRDS('/volume/afib/out/QC_preprocessing/CTRL_8_LA_postDoublet+RNAcontamFilt.rds')
d1 <- readRDS('/volume/afib/out/QC_preprocessing/D1_LA_postDoublet+RNAcontamFilt.rds')
d9 <- readRDS('/volume/afib/out/QC_preprocessing/D9_LA_postDoublet+RNAcontamFilt.rds')

af1 <- readRDS('/volume/afib/out/QC_preprocessing/AF_1_LA_postDoublet+RNAcontamFilt.rds')
af2 <- readRDS('/volume/afib/out/QC_preprocessing/AF_2_LA_postDoublet+RNAcontamFilt.rds')
af3 <- readRDS('/volume/afib/out/QC_preprocessing/AF_3_LA_postDoublet+RNAcontamFilt.rds')
af4 <- readRDS('/volume/afib/out/QC_preprocessing/AF_4_LA_postDoublet+RNAcontamFilt.rds')
af5 <- readRDS('/volume/afib/out/QC_preprocessing/AF_5_LA_postDoublet+RNAcontamFilt.rds')
af6 <- readRDS('/volume/afib/out/QC_preprocessing/AF_6_LA_postDoublet+RNAcontamFilt.rds')
af7 <- readRDS('/volume/afib/out/QC_preprocessing/AF_7_LA_postDoublet+RNAcontamFilt.rds')
af8 <- readRDS('/volume/afib/out/QC_preprocessing/AF_8_LA_postDoublet+RNAcontamFilt.rds')
hf6 <- readRDS('/volume/afib/out/QC_preprocessing/HF6_LA_postDoublet+RNAcontamFilt.rds')
hf9 <- readRDS('/volume/afib/out/QC_preprocessing/HF9_LA_postDoublet+RNAcontamFilt.rds')

rna_list <- c(ctrl1,ctrl2,ctrl3,ctrl4,ctrl5,ctrl6,ctrl7,ctrl8,d1,d9,
             af1,af2,af3,af4,af5,af6,af7,af8,hf6,hf9)

# Create combined object

In [ ]:
for (seurat in rna_list){
    DefaultAssay(seurat) <- 'ATAC'
}

In [ ]:
combined <- merge(rna_list[[1]], rna_list[2:length(rna_list)])

In [ ]:
# RNA analysis
DefaultAssay(combined) <- 'RNA'
combined <- NormalizeData(combined, verbose = FALSE)
combined <- FindVariableFeatures(combined, verbose = FALSE)
combined <- ScaleData(combined, verbose = FALSE)
combined <- RunPCA(combined, verbose = FALSE)
combined <- RunUMAP(combined, dims = 1:50, reduction.name = 'umap.rna', reduction.key = 'rnaUMAP_')

In [ ]:
# ATAC analysis
# We exclude the first dimension as this is typically correlated with sequencing depth
DefaultAssay(combined) <- 'ATAC'
combined <- RunTFIDF(combined)
combined <- FindTopFeatures(combined, min.cutoff='q0')
combined <- RunSVD(combined)
combined <- RunUMAP(combined, reduction='lsi', dims=2:50, reduction.name='umap.atac', reduction.key='atacUMAP_')

In [ ]:
# WNN multimodal analysis, representing a weighted combination of RNA and ATAC-seq modalities. 
combined <- FindMultiModalNeighbors(combined, reduction.list=list('pca', 'lsi'), dims.list=list(1:50, 2:50))
combined <- RunUMAP(combined, nn.name='weighted.nn', reduction.name='umap.wnn', reduction.key='wnnUMAP_')
combined <- FindClusters(combined, graph.name='wsnn', verbose=FALSE)

In [ ]:
options(repr.plot.width=20, repr.plot.height=7)
p1 <- DimPlot(combined, reduction = "umap.rna", group.by = "seurat_clusters", label = TRUE, repel = TRUE) + ggtitle("RNA")
p2 <- DimPlot(combined, reduction = "umap.atac", group.by = "seurat_clusters", label = TRUE, repel = TRUE) + ggtitle("ATAC")
p3 <- DimPlot(combined, reduction = "umap.wnn", group.by = "seurat_clusters", label = TRUE, repel = TRUE) + ggtitle("WNN")
p1 + p2 + p3 & NoLegend() & theme(plot.title = element_text(hjust = 0.5))

# Harmony

In [ ]:
library(harmony)

In [ ]:
# Harmonize based on RNA assay
combined <- RunHarmony(
    object = combined,
    group.by.vars = 'orig.ident',
    reduction = 'pca',
    assay.use = 'RNA',
    project.dim = FALSE,
    reduction.save = "harmony_rna"
)

In [ ]:
# Harmonize based on ATAC assay
combined <- RunHarmony(
    object = combined,
    group.by.vars = 'orig.ident',
    reduction = 'lsi',
    assay.use = 'ATAC',
    project.dim = FALSE,
    reduction.save = "harmony_atac"
)

In [ ]:
combined

In [ ]:
DefaultAssay(combined) <- 'ATAC'
combined <- RunUMAP(combined, reduction = 'harmony_atac', dims = 2:50, reduction.name = "umap.harmony.atac", reduction.key = "atacUMAPHarmony_")

In [ ]:
DefaultAssay(combined) <- 'RNA'
combined <- RunUMAP(combined, reduction = 'harmony_rna', dims = 1:50, reduction.name = "umap.harmony.rna", reduction.key = "rnaUMAPHarmony_")

In [ ]:
# rebuild a joint neighbor graph using both assays - WNN
combined <- FindMultiModalNeighbors(combined, reduction.list = list("harmony_rna", "harmony_atac"), dims.list = list(1:50, 2:50))
combined <- RunUMAP(combined, nn.name = "weighted.nn", reduction.name = "umap.harmony.wnn", reduction.key = "wnnUMAPHarmony_")
combined <- FindClusters(combined, graph.name = "wsnn")

In [ ]:
options(repr.plot.width=20, repr.plot.height=7)
p1 <- DimPlot(combined, reduction = "umap.harmony.rna", group.by = "condition", shuffle=TRUE) + ggtitle("RNA")
p2 <- DimPlot(combined, reduction = "umap.harmony.atac", group.by = "condition", shuffle=TRUE) + ggtitle("ATAC")
p3 <- DimPlot(combined, reduction = "umap.harmony.wnn", group.by = "condition", shuffle=TRUE) + ggtitle("WNN")
p1 + p2 + p3 & NoLegend() & theme(plot.title = element_text(hjust = 0.5))

In [ ]:
options(repr.plot.width=20, repr.plot.height=7)
p1 <- DimPlot(combined, reduction = "umap.harmony.rna", group.by = "seurat_clusters", label = TRUE, repel = TRUE) + ggtitle("RNA")
p2 <- DimPlot(combined, reduction = "umap.harmony.atac", group.by = "seurat_clusters", label = TRUE, repel = TRUE) + ggtitle("ATAC")
p3 <- DimPlot(combined, reduction = "umap.harmony.wnn", group.by = "seurat_clusters", label = TRUE, repel = TRUE) + ggtitle("WNN")
p1 + p2 + p3 & NoLegend() & theme(plot.title = element_text(hjust = 0.5))

In [ ]:
options(repr.plot.width=20, repr.plot.height=7)
p1 <- DimPlot(combined, reduction = "umap.harmony.rna", group.by = "orig.ident", shuffle=TRUE) + ggtitle("RNA")
p2 <- DimPlot(combined, reduction = "umap.harmony.atac", group.by = "orig.ident", shuffle=TRUE) + ggtitle("ATAC")
p3 <- DimPlot(combined, reduction = "umap.harmony.wnn", group.by = "orig.ident", shuffle=TRUE) + ggtitle("WNN")
p1 + p2 + p3 & NoLegend() & theme(plot.title = element_text(hjust = 0.5))

In [ ]:
saveRDS(combined, 'combined_multiome.rds')